## Image classification with Convolutional Neural Networks

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
PATH = "data/dogscats/"
sz=224
arch=vgg16
bs=64

In [4]:
# Uncomment the below if you need to reset your precomputed activations
# !rm -rf {PATH}tmp

In [4]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))

In [5]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [6]:
learn.fit(0.01, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[0.      0.07293 0.03327 0.98926]                             
[1.      0.0459  0.02831 0.99365]                             
[2.      0.05276 0.02811 0.99023]                             



In [7]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [8]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [9]:
learn.fit(1e-2, 2) # no data augmentation

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

[0.      0.07052 0.03116 0.99023]                             
[1.      0.04413 0.02376 0.99268]                             



In [10]:
learn.precompute=False

In [11]:
learn.fit(1e-2, 1, cycle_len=1) # data augmentation

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[0.      0.04587 0.02624 0.99121]                             



In [12]:
learn.unfreeze()

In [13]:
lr=np.array([1e-4,1e-3,1e-2])

In [14]:
learn.fit(lr, 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[0.      0.05122 0.02679 0.99121]                             



In [15]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2) # 1 + 2 + 4 = 7 epochs

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[0.      0.03452 0.02399 0.99072]                             
[1.      0.03264 0.01986 0.99072]                             
[2.      0.03629 0.01834 0.99072]                             
[3.      0.02966 0.01801 0.99121]                             
[4.      0.02366 0.01582 0.99268]                             
[5.      0.02498 0.01808 0.99072]                             
[6.      0.02876 0.01738 0.99219]                             



In [16]:
learn.fit(lr, 3, cycle_len=3) # 3 + 3 + 3 = 9 epochs

HBox(children=(IntProgress(value=0, description='Epoch', max=9), HTML(value='')))

[0.      0.02272 0.01559 0.99219]                             
[1.      0.02281 0.01561 0.9917 ]                             
[2.      0.01945 0.01533 0.99219]                             
[3.      0.01688 0.01517 0.99316]                             
[4.      0.02105 0.01574 0.99219]                             
[5.      0.01311 0.01607 0.99219]                             
[6.      0.01357 0.01877 0.99268]                             
[7.      0.01534 0.01589 0.99365]                             
[8.      0.01453 0.01802 0.9917 ]                             



In [20]:
log_preds,y = learn.TTA() # (5, 2000, 2), (2000,)
#accuracy(log_preds,y) # This does not work. (AttributeError: 'bool' object has no attribute 'mean')
accuracy(np.mean(log_preds, axis=0), y)

0.9955

In [21]:
??accuracy()

# def accuracy(preds, targs):
#    preds = np.argmax(preds, axis=1)
#    return (preds==targs).mean()